<a href="https://colab.research.google.com/github/SivaGanesh-A/scratch/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pinecone-client scikit-learn numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.3 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
from pinecone import Pinecone, ServerlessSpec
from sklearn.neighbors import NearestNeighbors

# Initialize Pinecone Client
pc = Pinecone(api_key="pcsk_7CbPGf_MiYtwtTXpvdspQBLWJ2GdDrCkduD6mmt8jnF5nGTQU31HDSV4RAvEttsnCKg3a4")

# Connect to Index
index_name = "sampleindex"
index = pc.Index(index_name)


In [4]:
def query_pinecone(query_vector, top_k=3):
    # Query Pinecone index to find nearest vectors
    result = index.query(
        vector=query_vector,  # The query vector
        top_k=top_k,  # Number of nearest neighbors to return
        include_values=True  # Include the actual vector data
    )
    return result


In [5]:
# Prepare data for training
# You can use your vectors from the Pinecone index or other dataset
vector_data_sets = [
    [
        [0.0, 51.77, 1.0],
        [1.0, 53.74, 1.0],
        [2.0, 50.1, 1.0],
        [3.0, 47.82, 1.0],
        [4.0, 54.23, 1.0],
        [5.0, 22.65, 1.0],
    ],
    [
        [0.0, 33.34, 1.0],
        [1.0, 17.93, 1.0],
        [2.0, 22.6, 1.0],
        [3.0, 52.15, 1.0],
        [4.0, 13.99, 1.0],
        [5.0, 20.66, 1.0],
    ],
    [
        [0.0, 17.46, 1.0],
        [1.0, 52.29, 1.0],
        [2.0, 35.02, 1.0],
        [3.0, 57.25, 1.0],
        [4.0, 23.88, 1.0],
        [5.0, 37.59, 1.0],
    ],
    [
        [0.0, 51.33, 1.0],
        [1.0, 45.17, 1.0],
    ],
]

# Flatten and label the data (for supervised learning)
vectors = []
labels = []
for i, dataset in enumerate(vector_data_sets):
    for j, vector in enumerate(dataset):
        vectors.append(vector)
        labels.append(i)  # Labeling vectors by their dataset index

vectors = np.array(vectors)
labels = np.array(labels)

# Train a Nearest Neighbor Model
model = NearestNeighbors(n_neighbors=3, metric='euclidean')
model.fit(vectors)

# Test the Model
query_vector = np.array([1.0, 52.0, 1.0])  # Example query vector
distances, indices = model.kneighbors([query_vector])

print(f"Nearest neighbors for the query vector {query_vector}:")
for dist, idx in zip(distances[0], indices[0]):
    print(f"Vector: {vectors[idx]} | Distance: {dist} | Label: {labels[idx]}")


Nearest neighbors for the query vector [ 1. 52.  1.]:
Vector: [ 1.   52.29  1.  ] | Distance: 0.28999999999999915 | Label: 2
Vector: [ 0.   51.77  1.  ] | Distance: 1.0261091559868272 | Label: 0
Vector: [ 0.   51.33  1.  ] | Distance: 1.203702621082135 | Label: 3


In [7]:
import numpy as np

# Example of querying Pinecone and using the NN model
query_vector = np.array([1.0, 52.0, 1.0])  # Example query vector

# Convert the NumPy array to a list
query_vector_list = query_vector.tolist()

# Now call the query function with the list
pinecone_results = query_pinecone(query_vector_list)

print("Pinecone Query Results:")
print(pinecone_results)

Pinecone Query Results:
{'matches': [{'id': 'vec_2_1',
              'score': 1.38671875,
              'values': [1.0, 52.29, 1.0]},
             {'id': 'vec_0_0',
              'score': 1.71777344,
              'values': [0.0, 51.77, 1.0]},
             {'id': 'vec_3_0',
              'score': 2.12548828,
              'values': [0.0, 51.33, 1.0]}],
 'namespace': '',
 'usage': {'read_units': 6}}


In [8]:
import numpy as np

# Function to test the trained nearest neighbor model
def test_model(query_vector):
    distances, indices = model.kneighbors([query_vector])
    print(f"Query Vector: {query_vector}")
    print("Nearest Neighbors:")
    for dist, idx in zip(distances[0], indices[0]):
        print(f"Vector: {vectors[idx]} | Distance: {dist} | Label: {labels[idx]}")

# Example query
query_vector = np.array([1.0, 52.0, 1.0])
test_model(query_vector)


Query Vector: [ 1. 52.  1.]
Nearest Neighbors:
Vector: [ 1.   52.29  1.  ] | Distance: 0.28999999999999915 | Label: 2
Vector: [ 0.   51.77  1.  ] | Distance: 1.0261091559868272 | Label: 0
Vector: [ 0.   51.33  1.  ] | Distance: 1.203702621082135 | Label: 3
